In [ ]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel('Questions with labels.xlsx')

# 创建一个空的DataFrame来保存结果
result_df = pd.DataFrame(columns=['Question','Question_type', 'Entity1:Object','Entity2:Spectrum'])

# 遍历原始DataFrame的每一行
for index, row in df.iterrows():
    if row[2] == 'Randow questions':
        # 如果第三列的值为"Spectral Detection Method"，则提取第二列和第四列的值
        new_row = pd.DataFrame({'Question': [row[1]], 'Question_type':[row[2]],'Entity1:Object': [row[3]],,'Entity2:Spectrum':[row[4]]})
        result_df = pd.concat([result_df, new_row], ignore_index=True)

# 保存结果到新的Excel文件
result_df.to_excel('Answers_type3.xlsx', index=False)

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 读取Excel文件
df1 = pd.read_excel('Answers_type3.xlsx')
df2 = pd.read_excel("Label.xlsx")

# 定义TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

# 准备语料库
corpus = [str(row[5]) + " " + str(row[6]) + " " + str(row[14]).lower() for _, row in df2.iterrows()]

# 计算TF-IDF矩阵
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 定义余弦相似度检索函数
def cosine_retrieval(keywords, dataframe, top_n=1):
    # 将关键词转换为小写
    keywords = keywords.strip().lower()
    
    # 构建查询文本的TF-IDF向量
    query_vector = tfidf_vectorizer.transform([keywords])
    
    # 计算查询文本与语料库中每个文档的余弦相似度
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # 获取相似度最高的top_n行的索引
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 返回相关度最高的论文标题
    return dataframe.iloc[top_indices][dataframe.columns[0]]

# 遍历df1中每行的第三列值作为关键词进行检索
results = []
for index, row in df1.iterrows():
    keywords = str(row[2]) + " " + str(row[3])  # 第三、四列的索引
    relevant_rows = cosine_retrieval(keywords, df2)
    results.append(relevant_rows.values)

# 将结果转换为DataFrame并与df1合并
results_df = pd.DataFrame(results, columns=['Related paper'])
df1 = pd.concat([df1, results_df], axis=1)

# 保存更新后的DataFrame到新的Excel文件
df1.to_excel('Answers_type3.xlsx', index=False)

print("处理完成，更新后的文件已保存为 'Answers_type3.xlsx'")


C:\Users\Raytine\AppData\Local\Temp\ipykernel_6752\1843390087.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  keywords = str(row[2]) + " " + str(row[3])  # 第三、四列的索引
C:\Users\Raytine\AppData\Local\Temp\ipykernel_6752\1843390087.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  corpus = [str(row[5]) + " " + str(row[11]) + " " + str(row[14])+" "+str(row[6]).lower() for _, row in dataframe.iterrows()]


In [6]:
import os  
import pandas as pd  
from langchain.vectorstores import FAISS  
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  
from langchain.chains import RetrievalQA  
from langchain.chat_models import ChatOpenAI  
from langchain import PromptTemplate  
  
# 设置Chat-GPT API的密钥  
OPENAI_KEY = 'your key'  
  
# 加载Chat-GPT模型  
gpt_model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1, max_tokens=512, api_key=OPENAI_KEY)  
  
# 论文文件夹路径  
paper_folder = "F:\Code\Word_detecting\Vector_ku\Save_Vector_Store" 
  
# Excel文件路径  
excel_path = 'Answers_type3.xlsx'  # 替换为您的Excel文件路径  

# 加载Excel文件  
df = pd.read_excel(excel_path)  
  
# 用于保存答案的列表  
answers = []  
  
# Prompt模板  
prompt_template = """Based on the information of the faiss files of the related papers, briefly extract information from the text that is relevant to the question being asked.  
  
            known information:  
            {context}  
  
            Question:  
            {question}"""  
  
# 遍历Excel文件的每一行  
for index, row in df.iterrows():  
    paper_title = str(row.iloc[5])  # 假设第六列是论文标题  
    query_question = row.iloc[0]  # 假设第一列是查询问题  
      
    # 在文件夹中查找对应的.faiss文件
    vs_path = os.path.join(paper_folder, paper_title, 'Vector')
    faiss_path = [f for f in os.listdir(vs_path) if f.endswith('.faiss')]

    if faiss_path:  
        # 加载向量存储  
        vector_store = FAISS.load_local(vs_path,
                                    HuggingFaceEmbeddings(model_name="BAAI/bge-small-en"),
                                    index_name=paper_title,
                                    allow_dangerous_deserialization=True)
          
        # 创建RetrievalQA链  
        knowledge_chain = RetrievalQA.from_llm(llm=gpt_model,
                                           retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                           prompt=PromptTemplate(
                                               template=prompt_template,
                                               input_variables=["context", "question"]))
        result = knowledge_chain({"query": query_question})
        generated_question = result['result']
        print(paper_title,generated_question)
        answers.append(generated_question)  
    else:  
        # 如果找不到.faiss文件，添加一个占位符或错误信息  
        answers.append("No FAISS file found for this paper.")  
  
# 将答案添加到DataFrame中  
df['Answer'] = answers  
  
# 保存修改后的DataFrame回Excel文件  
df.to_excel(excel_path, index=False)

1 The prediction of Chinese black tea ranking can be improved by incorporating different machine learning algorithms with Fourier transform near-infrared reflectance spectroscopy. In the related papers, it was shown that cognitive spectroscopy, which combines near-infrared spectroscopy with machine learning and evolutionary algorithms, can be utilized as a rapid strategy to identify Chinese black tea grades. The study used feature selection evolutionary algorithms such as genetic algorithm, simulated annealing, and particle swarm optimization to search for the best preprocessed characteristic wavelengths. Cognitive models of Chinese black tea ranks were constructed using extreme learning machine, K-nearest neighbor, and support vector machine methods based on the selected characteristic variables. The PSO-SVM model showed the best predictive performance with high correlation coefficients, root mean square error, and correct discriminant rate. This approach demonstrated that incorporati

PermissionError: [Errno 13] Permission denied: 'Answers_type3.xlsx'

In [7]:
df.to_excel(excel_path, index=False)

In [9]:
import os  
import pandas as pd  
from langchain.vectorstores import FAISS  
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  
from langchain.chains import RetrievalQA  
from langchain.chat_models import ChatOpenAI  
from langchain import PromptTemplate  
  
# 设置Chat-GPT API的密钥  
OPENAI_KEY = 'your_key'  
  
# 加载Chat-GPT模型  
gpt_model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1, max_tokens=512, api_key=OPENAI_KEY)  
  
# 论文文件夹路径  
paper_folder = "F:\Code\Word_detecting\Vector_ku\Save_Vector_Store" 
  
# Excel文件路径  
excel_path = 'Answers_type3.xlsx'  # 替换为您的Excel文件路径  

# 加载Excel文件  
df = pd.read_excel(excel_path)  
  
# 用于保存答案的列表  
answers = []  
  

  
# Prompt模板  
prompt_template = """Based on the information of the faiss files of the related papers, answer the user's questions concisely and professionally.  
  
            known information:  
            {context}  
  
            Question:  
            {question}"""  
  
# 遍历Excel文件的每一行  
for index, row in df.iterrows():  
    paper_title = str(row.iloc[5])  # 假设第六列是论文标题  
    query_question = row.iloc[0]  # 假设第一列是查询问题  
      
    # 在文件夹中查找对应的.faiss文件
    vs_path = os.path.join(paper_folder, paper_title, 'Vector')
    faiss_path = [f for f in os.listdir(vs_path) if f.endswith('.faiss')]

    if faiss_path:  
        # 加载向量存储  
        vector_store = FAISS.load_local(vs_path,
                                    HuggingFaceEmbeddings(model_name="BAAI/bge-small-en"),
                                    index_name=paper_title,
                                    allow_dangerous_deserialization=True)
          
        # 创建RetrievalQA链  
        knowledge_chain = RetrievalQA.from_llm(llm=gpt_model,
                                           retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                           prompt=PromptTemplate(
                                               template=prompt_template,
                                               input_variables=["context", "question"]))
        result = knowledge_chain({"query": query_question})
        generated_question = result['result']
        print(paper_title,generated_question)
        answers.append(generated_question)  
    else:  
        # 如果找不到.faiss文件，添加一个占位符或错误信息  
        answers.append("No FAISS file found for this paper.")  
  
# 将答案添加到DataFrame中  
df['Answer1'] = answers  
  
# 保存修改后的DataFrame回Excel文件  
df.to_excel(excel_path, index=False)

No sentence-transformers model found with name BAAI/bge-small-en. Creating a new one with MEAN pooling.


1 The prediction of Chinese black tea ranking can be improved by incorporating different machine learning algorithms with Fourier transform near-infrared reflectance spectroscopy through the utilization of cognitive spectroscopy combined with evolutionary algorithms. This involves applying various spectral pretreating methods, such as standard normal variate transformation, multiplicative scatter correction, and min/max normalization, to eliminate noise and baseline drift. Additionally, employing evolutionary algorithms like genetic algorithm, simulated annealing, and particle swarm optimization can help develop preferred characteristic variable modeling algorithms for enhanced learning efficiency. Finally, utilizing machine learning approaches like extreme learning machine, K-nearest neighbor, and support vector machine can aid in building cognitive models based on optimized characteristic wavelengths for qualitative analysis of Chinese black tea quality ranks.
6 The Bayesian-based de

In [1]:
import pandas as pd
import json
# 加载Excel文件
file_path = 'Answers_type3.xlsx'  # 请根据实际情况修改文件路径
sheet_name = 'Sheet1'  # 根据实际情况修改工作表名称
df = pd.read_excel(file_path, sheet_name=sheet_name)
# 准备JSON数据
json_data = []
for index, row in df.iterrows():
    question = row['Question']
    question_type = row['Question_type']  # 假设第二列是问题类型
    entity1 = row['Entity1:Object']  # 假设第三列是实体对象
    entity2=row['Entity2:Spectrum']
    Knowledge=row['Knowledge']
    answer = row['Answer']

    # 构造单个JSON对象
    json_object = {
        "Instruction":'Based on the information of Knowledge, Entity1:Object and Entity2:Spectrum in this section, answer the question concisely and professionally.',
        "Question": question,
        "Question_type": question_type,
        "Entity1:Object": entity1,
        "Entity2:Spectrum":entity2,
        "Knowledge":Knowledge,
        "Output": answer
    }
    json_data.append(json_object)

# 将数据输出为JSON格式
json_output = json.dumps(json_data, indent=4)
print(json_output)

# 如果需要，可以将JSON数据保存到文件中
with open('Answers_type3.json', 'w') as json_file:
    json_file.write(json_output)

[
    {
        "Instruction": "Based on the information of Knowledge, Entity1:Object and Entity2:Spectrum in this section, answer the question concisely and professionally.",
        "Question": "How can the prediction of Chinese black tea ranking be improved by incorporating different machine learning algorithms with Fourier transform near-infrared reflectance spectroscopy?",
        "Question_type": "Random questions",
        "Entity1:Object": "Chinese black tea",
        "Entity2:Spectrum": "near-infrared spectroscopy(nir)",
        "Knowledge": "P1. Purpose: The purpose of the study was to develop a robust model for grading Chinese black tea using cognitive spectroscopy. ###\n\nP2. Methods: The study used near-infrared spectroscopy (NIRS) combined with machine learning and evolutionary algorithms to analyze the spectra of Chinese black tea samples. The spectra were preprocessed using standard normal variate transformation (SNV), multiplicative scatter correction (MSC), and minimu

In [1]:
import json
import random

# 假设三个JSON文件的路径
file_paths = [ 'Answers_type2.json', 'Answers_type3.json']

# 用于存储合并后的数据
all_data = []

# 读取每个文件并将数据添加到列表中
for path in file_paths:
    with open(path, 'r') as file:
        data = json.load(file)
        all_data.extend(data)  # 假设每个文件中的数据都是列表形式

# 随机重新排列合并后的数据
random.shuffle(all_data)

# 将随机排列后的数据写入新的JSON文件
with open('Answers_typeall.json', 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

print("数据已合并并随机排列，保存到 'merged_randomized.json' 文件中。")


数据已合并并随机排列，保存到 'merged_randomized.json' 文件中。
